# Domain Name System

---

## Table of Contents
```{contents}
```

---

## `dig`

```bash
dig cloudflare.com
dig cloudflare.com +short
dig cloudflare.com +short +dnssec # the last line of output is the RRSIG record which a DNS resolver uses to determine whether a DNS response is trusted
dig DNSKEY cloudflare.com +short  # retrieve the public key used to verify the DNS record
```
```
257 3 13 mdsswUyr3DPW132mOi8V9xESWE8jTo0dxCjjnopKl+GqJxpVXckHAeF+ KkxLbxILfDLUT0rAK9iUzy1L53eKGQ== # zone-signing key (ZSK)
256 3 13 oJMRESz5E4gYzS/q6XDrvU1qMPYIjCWzJaOau8XNEZeqCYKD5ar0IRd8 KqXXFJkqmVfRvMGPmM1x8fGAa2XhSA== # key-signing key (KSK)
```
```bash
dig DS cloudflare.com +short # find a DS record
dig DS cloudflare.com +trace # dig confirms whether an answer is returned by the nameserver for cloudflare.com or the nameserver for .com
```

---

## Unbound

### Config

```
/etc/unbound/unbound.conf
/etc/unbound/unbound.conf.d/pi-hole.conf
```

```
server:
    # can be uncommented if you do not need user privilige protection
    # username: ""

    # can be uncommented if you do not need file access protection
    # chroot: ""

    # location of the trust anchor file that enables DNSSEC. note that
    # the location of this file can elsewhere
    auto-trust-anchor-file: "/usr/local/etc/unbound/root.key"
    # auto-trust-anchor-file: "/var/lib/unbound/root.key"

    gg# send minimal amount of information to upstream servers to enhance privacy
    qname-minimisation: yes

    # addresses from the IP range that are allowed to connect to the resolver
    # access-control: 192.168.0.0/16 allow
    # access-control: 2001:DB8/64 allow

    # If no logfile is specified, syslog is used
    logfile: "/var/log/unbound/unbound.log"
    verbosity: 2

    # specify the interfaces to answer queries from by ip-address.
    # interface: 0.0.0.0
    # interface: ::0
    interface: 127.0.0.1
    port: 5335
    do-ip4: yes
    do-udp: yes
    do-tcp: yes

    # May be set to yes if you have IPv6 connectivity
    do-ip6: no

    # You want to leave this to no unless you have *native* IPv6. With 6to4 and
    # Terredo tunnels your web browser should favor IPv4 for the same reasons
    prefer-ip6: no

    # Use this only when you downloaded the list of primary root servers!
    # If you use the default dns-root-data package, unbound will find it automatically
    root-hints: "/var/lib/unbound/root.hints"

    # Trust glue only if it is within the server's authority
    harden-glue: yes

    # Require DNSSEC data for trust-anchored zones, if such data is absent, the zone becomes BOGUS
    harden-dnssec-stripped: yes

    # Don't use Capitalization randomization as it known to cause DNSSEC issues sometimes
    # see https://discourse.pi-hole.net/t/unbound-stubby-or-dnscrypt-proxy/9378 for further details
    use-caps-for-id: no

    # Reduce EDNS reassembly buffer size.
    # IP fragmentation is unreliable on the Internet today, and can cause
    # transmission failures when large DNS messages are sent via UDP. Even
    # when fragmentation does work, it may not be secure; it is theoretically
    # possible to spoof parts of a fragmented DNS message, without easy
    # detection at the receiving end. Recently, there was an excellent study
    # >>> Defragmenting DNS - Determining the optimal maximum UDP response size for DNS <<<
    # by Axel Koolhaas, and Tjeerd Slokker (https://indico.dns-oarc.net/event/36/contributions/776/)
    # in collaboration with NLnet Labs explored DNS using real world data from the
    # the RIPE Atlas probes and the researchers suggested different values for
    # IPv4 and IPv6 and in different scenarios. They advise that servers should
    # be configured to limit DNS messages sent over UDP to a size that will not
    # trigger fragmentation on typical network links. DNS servers can switch
    # from UDP to TCP when a DNS response is too big to fit in this limited
    # buffer size. This value has also been suggested in DNS Flag Day 2020.
    edns-buffer-size: 1232

    # Perform prefetching of close to expired message cache entries
    # This only applies to domains that have been frequently queried
    prefetch: yes

    # One thread should be sufficient, can be increased on beefy machines. In reality for most users running on small networks or on a single machine, it should be unnecessary to seek performance enhancement by increasing num-threads above 1.
    num-threads: 1

    # Ensure kernel buffer is large enough to not lose messages in traffic spikes
    so-rcvbuf: 1m

    # Ensure privacy of local IP ranges
    private-address: 192.168.0.0/16
    private-address: 169.254.0.0/16
    private-address: 172.16.0.0/12
    private-address: 10.0.0.0/8
    private-address: fd00::/8
    private-address: fe80::/10

    # Remote control config section.
    remote-control:
        # Enable remote control with unbound-control(8) here.
        # set up the keys and certificates with unbound-control-setup.
        control-enable: yes

        # Set to no and use an absolute path as control-interface to use
        # a unix local named pipe for unbound-control.
        control-use-cert: no
```

### Test

```bash
unbound-checkconf # unbound-checkconf: no errors in /etc/unbound/unbound.conf

sudo service unbound status
sudo service unbound restart

sudo grep -v '#\|^$' -R /etc/unbound/unbound.conf.d
sudo grep -v '#\|^$' -R /etc/dhcpcd.conf
sudo grep -v '#\|^$' -R /etc/resolvconf.conf
sudo cat /var/cache/unbound/resolvconf_resolvers.conf
```

Test Unbound

```bash
sudo service unbound restart
dig pi-hole.net @127.0.0.1 -p 5335
```

Test DNSSEC

```bash
dig dnssec.works @127.0.0.1 -p 5335
```
```
; <<>> DiG 9.11.5-P4-5.1+deb10u9-Raspbian <<>> dnssec.works @127.0.0.1 -p 5335
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 24162
;; flags: qr rd ra ad; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 1232
;; QUESTION SECTION:
;dnssec.works.			IN	A

;; ANSWER SECTION:
dnssec.works.		3600	IN	A	5.45.107.88

;; Query time: 104 msec
;; SERVER: 127.0.0.1#5335(127.0.0.1)
;; WHEN: Tue Nov 28 23:31:11 EST 2023
;; MSG SIZE  rcvd: 57
```
```bash
dig fail01.dnssec.works @127.0.0.1 -p 5335
```
```
; <<>> DiG 9.11.5-P4-5.1+deb10u9-Raspbian <<>> fail01.dnssec.works @127.0.0.1 -p 5335
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: SERVFAIL, id: 33492
;; flags: qr rd ra; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 1232
;; QUESTION SECTION:
;fail01.dnssec.works.		IN	A

;; Query time: 0 msec
;; SERVER: 127.0.0.1#5335(127.0.0.1)
;; WHEN: Tue Nov 28 23:31:51 EST 2023
;; MSG SIZE  rcvd: 48
```

### More

```bash
sudo unbound-control status
sudo unbound-control lookup pi-hole.net
```

---

## Resources

* [ [h](https://dnscrypt.info) ][ d ][ g ] DNSCrypt
* [ [h](https://dnsviz.net/) ] DNSViz
* [ [h](https://pi-hole.net) ][ [d](https://docs.pi-hole.net) ][ [g](https://github.com/pi-hole) ] Pi-hole
  * [The Big Blocklist Collection](https://firebog.net)
  * Pi-hole on UDM [run-pihole](https://github.com/boostchicken/udm-utilities/tree/master/run-pihole) [on-boot-script](https://github.com/boostchicken/udm-utilities/tree/master/on-boot-script)
  * [ [article](https://jekhokie.github.io/unifi/pihole/networking/2020/12/15/unifi-pihole-ad-blocking-with-failover.html) ] "Pi-hole Ad-Blocking with Failover on UniFi".
  * [ [article](https://reposhub.com/linux/system-utilities/tusc-wireguard-kmod.html) ] "WireGuard for UDM series routers".
* [ h ][ [d](https://unbound.docs.nlnetlabs.nl/en/latest/) ][ [g](https://github.com/NLnetLabs/unbound) ] Unbound
  * https://docs.pi-hole.net/guides/dns/unbound/

* [ [d](https://adguard-dns.io/kb/) ] Adguard - DNS
* [ [d](https://www.cloudflare.com/learning/dns/what-is-dns/) ] Cloudflare - What is DNS?
* [ [d](https://www.cloudflare.com/learning/dns/glossary/dynamic-dns/) ] Cloudflare - DDNS
* [ [d](https://help.firewalla.com/hc/en-us/articles/4570608120979-Firewalla-DNS-Services-Introduction) ] Firewalla - DNS Services
* [ [d](https://help.firewalla.com/hc/en-us/articles/360038449734-DNS-over-HTTPS) ] Firewalla - DoH
* [ [d](https://meshnet.nordvpn.com/how-to/security/protect-network-with-adguard-home) ] NordVPN - AdGuard Home

online
* [ [o](https://superuser.com/questions/570082/in-etc-resolv-conf-what-exactly-does-the-search-configuration-option-do) ] StackExchange. "In /etc/resolv.conf, what exactly does the "search" configuration option do?".

---

## Terms

* [ w ] BIND
* [ [w](https://en.wikipedia.org/wiki/DNS_over_HTTPS) ] DNS over HTTPS (DoH)
* [ [w](https://en.wikipedia.org/wiki/DNS_over_TLS) ] DNS over TLS (DoT)
* [ [w](https://en.wikipedia.org/wiki/DNS_root_zone) ] DNS Root Zone
* [ w ] DNS Shield
* [ w ] DNS Sinkhole
* [ [w](https://en.wikipedia.org/wiki/DNS_zone) ] DNS Zone
* [ [w](https://en.wikipedia.org/wiki/DNSCrypt) ] DNSCrypt
* [ w ] Domain Controller
* [ [w](https://en.wikipedia.org/wiki/Domain_name) ] Domain Name
* [ [w](https://en.wikipedia.org/wiki/Domain_Name_System) ] Domain Name System (DNS)
* [ [w](https://en.wikipedia.org/wiki/Domain_Name_System_Security_Extensions) ] Domain Name System Security Extensions (DNSSEC)
* [ w ] Dynamic DNS (DDNS)
* [ [w](https://en.wikipedia.org/wiki/Fully_qualified_domain_name) ] Fully-Qualified Domain Name (FQDN)
* [ [w](https://en.wikipedia.org/wiki/ICANN) ] Internet Corporation for Assigned Names and Numbers (ICANN)
* [ w ] Key-Signing Key (KSK)
  * used to verify the signatures of the DNSKEY, CDS, and CDNSKEY records
* [ [w](https://en.wikipedia.org/wiki/Name_server) ] Name Server
* [ [w](https://en.wikipedia.org/wiki/Public_recursive_name_server) ] Public DNS Resolver
* [ [w](https://en.wikipedia.org/wiki/Public_recursive_name_server) ] Public Recursive Name Server
* [ [w](https://en.wikipedia.org/wiki/Root_name_server) ] Root Name Server
* [ [w](https://en.wikipedia.org/wiki/Search_domain) ] Search Domain
* [ [w](https://en.wikipedia.org/wiki/Top-level_domain) ] Top-Level Domain (TLD)
* [ w ] Zone-Signing Key (ZSK)
  * used to verify the DNS record signatures for A, MX, CNAME, SRV, etc.

---

## Notes

Static DNS vs DDNS
* DDNS - the automatic process of updating the server list within a domain name server with the server's IP address and FQDN

---